# LLMalMorph Test trên Kaggle

Notebook này test hệ thống LLMalMorph với C.rar và CPP.rar trên Kaggle environment.

## 📋 Hướng Dẫn

1. **Upload RAR files**: Upload `C.rar` và `CPP.rar` vào Kaggle dataset hoặc input folder
2. **Set API Key**: Thay `your-mistral-api-key-here` bằng API key thật của bạn ở cell 2
3. **Run All Cells**: Chạy tất cả cells để test hệ thống


## 1. Install Dependencies


In [ ]:
# Install dependencies
!pip install -q mistralai requests tree-sitter tree-sitter-c tree-sitter-cpp rarfile

print("✓ Dependencies installed")


## 2. Setup Environment & API Key


In [ ]:
# Setup environment
import os
import sys
from pathlib import Path

# ⚠️ QUAN TRỌNG: Thay "your-mistral-api-key-here" bằng API key thật của bạn
MISTRAL_API_KEY = "your-mistral-api-key-here"  # ← THAY ĐỔI Ở ĐÂY

if MISTRAL_API_KEY != "your-mistral-api-key-here":
    os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY
    print("✓ API Key set")
else:
    print("⚠️  WARNING: API Key chưa được set!")
    print("   Vui lòng thay 'your-mistral-api-key-here' bằng API key thật")

# Add src to path
if os.path.exists('/kaggle/working/LLMalMorph'):
    sys.path.insert(0, '/kaggle/working/LLMalMorph/src')
    print("✓ Path configured")
else:
    print("⚠️  LLMalMorph repository not found. Will clone in next cell.")


## 3. Clone Repository


In [ ]:
# Clone repository nếu chưa có
if not os.path.exists('/kaggle/working/LLMalMorph'):
    print("Cloning repository...")
    !git clone https://github.com/AJAkil/LLMalMorph.git /kaggle/working/LLMalMorph
    print("✓ Repository cloned")
    
    # Add src to path
    sys.path.insert(0, '/kaggle/working/LLMalMorph/src')
else:
    print("✓ Repository already exists")
    sys.path.insert(0, '/kaggle/working/LLMalMorph/src')


## 4. Build Tree-sitter


In [ ]:
# Build tree-sitter từ source (nếu cần)
from tree_sitter import Language

# Try to use pre-built libraries first
try:
    import tree_sitter_c as ts_c
    import tree_sitter_cpp as ts_cpp
    C_LANGUAGE = Language(ts_c.language())
    CPP_LANGUAGE = Language(ts_cpp.language())
    print("✓ Using pre-built tree-sitter libraries")
except ImportError:
    print("Pre-built libraries not available, building from source...")
    
    # Clone tree-sitter-c nếu chưa có
    if not os.path.exists('tree-sitter-c'):
        !git clone --branch v0.20.2 https://github.com/tree-sitter/tree-sitter-c.git
        print("✓ Cloned tree-sitter-c")
    
    # Build library
    os.makedirs('build', exist_ok=True)
    
    if not os.path.exists('build/my-languages.so'):
        try:
            Language.build_library(
                'build/my-languages.so',
                ['tree-sitter-c']
            )
            print("✓ Built tree-sitter library")
        except Exception as e:
            print(f"⚠️  Build failed: {e}")
            print("   Will try to use pre-built libraries")
    else:
        print("✓ Tree-sitter library already exists")


## 5. Extract RAR Files


In [ ]:
# Tìm và extract RAR files
import rarfile

# Tìm RAR files ở các vị trí có thể
possible_c_paths = [
    "/kaggle/working/C.rar",
    "/kaggle/input/c-archive/C.rar",
    "/kaggle/input/c/C.rar",
    "/kaggle/input/C.rar",
]

possible_cpp_paths = [
    "/kaggle/working/CPP.rar",
    "/kaggle/input/cpp-archive/CPP.rar",
    "/kaggle/input/cpp/CPP.rar",
    "/kaggle/input/CPP.rar",
]

# Tìm C.rar
c_rar_path = None
for path in possible_c_paths:
    if os.path.exists(path):
        c_rar_path = path
        break

# Tìm CPP.rar
cpp_rar_path = None
for path in possible_cpp_paths:
    if os.path.exists(path):
        cpp_rar_path = path
        break

# Extract C.rar
if c_rar_path:
    extract_c_dir = "/kaggle/working/extracted_C"
    os.makedirs(extract_c_dir, exist_ok=True)
    
    try:
        with rarfile.RarFile(c_rar_path) as rf:
            rf.extractall(extract_c_dir)
        print(f"✓ Extracted C.rar from {c_rar_path}")
        print(f"  → {extract_c_dir}")
    except Exception as e:
        print(f"✗ Failed to extract C.rar: {e}")
else:
    print("⚠️  C.rar not found. Please upload to:")
    print("   - /kaggle/working/C.rar")
    print("   - Or add as Kaggle dataset")

# Extract CPP.rar
if cpp_rar_path:
    extract_cpp_dir = "/kaggle/working/extracted_CPP"
    os.makedirs(extract_cpp_dir, exist_ok=True)
    
    try:
        with rarfile.RarFile(cpp_rar_path) as rf:
            rf.extractall(extract_cpp_dir)
        print(f"✓ Extracted CPP.rar from {cpp_rar_path}")
        print(f"  → {extract_cpp_dir}")
    except Exception as e:
        print(f"✗ Failed to extract CPP.rar: {e}")
else:
    print("⚠️  CPP.rar not found. Please upload to:")
    print("   - /kaggle/working/CPP.rar")
    print("   - Or add as Kaggle dataset")


In [ ]:
# Tìm tất cả C và CPP files
def find_source_files(directory, extensions):
    """Tìm source files trong directory"""
    directory = Path(directory)
    if not directory.exists():
        return []
    
    files = []
    for ext in extensions:
        files.extend(directory.rglob(f'*{ext}'))
    
    return sorted(files)

# Tìm C files
c_files = []
if os.path.exists("/kaggle/working/extracted_C"):
    c_files = find_source_files("/kaggle/working/extracted_C", ['.c'])
    print(f"✓ Found {len(c_files)} C files")
    if c_files:
        print(f"  First file: {c_files[0].name}")

# Tìm CPP files
cpp_files = []
if os.path.exists("/kaggle/working/extracted_CPP"):
    cpp_files = find_source_files("/kaggle/working/extracted_CPP", ['.cpp', '.cxx', '.cc'])
    print(f"✓ Found {len(cpp_files)} CPP files")
    if cpp_files:
        print(f"  First file: {cpp_files[0].name}")


## 7. Import Hệ Thống


In [ ]:
# Import hệ thống
from automation import IntegratedPipeline
from config import get_config, setup_logging
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
config = get_config()
setup_logging(config)

print("✓ System imported successfully")


## 8. Test C File


In [ ]:
# Test với một C file
if c_files:
    test_file = c_files[0]
    print(f"\n{'='*60}")
    print(f"Testing: {test_file.name}")
    print(f"{'='*60}")
    
    # Read file
    with open(test_file, 'r', encoding='utf-8', errors='ignore') as f:
        source_code = f.read()
    
    # Initialize pipeline
    try:
        pipeline = IntegratedPipeline(
            language='c',
            llm_model='codestral-2508',
            api_key=os.environ.get('MISTRAL_API_KEY'),
            max_fix_attempts=1,
        )
        
        # Process
        results = pipeline.process_variant(
            source_file=str(test_file),
            variant_code=source_code,
            original_code=source_code,
            auto_fix=False,  # Skip auto-fix for quick test
            run_tests=False,  # Skip compilation test (no compiler on Kaggle)
        )
        
        # Print results
        comp = results.get('compilation', {})
        quality = results.get('quality', {})
        
        print(f"\nResults:")
        print(f"  Quality Score: {quality.get('quality_score', 0):.2f}")
        print(f"  Syntax Valid: {'✓' if quality.get('syntax_valid') else '✗'}")
        print(f"  Security Issues: {len(quality.get('security_issues', []))}")
        print(f"  Overall: {'✓ SUCCESS' if results.get('success') else '✗ FAILED'}")
        
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️  No C files found")


## 9. Test CPP File


In [ ]:
# Test với một CPP file
if cpp_files:
    test_file = cpp_files[0]
    print(f"\n{'='*60}")
    print(f"Testing: {test_file.name}")
    print(f"{'='*60}")
    
    # Read file
    with open(test_file, 'r', encoding='utf-8', errors='ignore') as f:
        source_code = f.read()
    
    # Initialize pipeline
    try:
        pipeline = IntegratedPipeline(
            language='cpp',
            llm_model='codestral-2508',
            api_key=os.environ.get('MISTRAL_API_KEY'),
            max_fix_attempts=1,
        )
        
        # Process
        results = pipeline.process_variant(
            source_file=str(test_file),
            variant_code=source_code,
            original_code=source_code,
            auto_fix=False,
            run_tests=False,
        )
        
        # Print results
        comp = results.get('compilation', {})
        quality = results.get('quality', {})
        
        print(f"\nResults:")
        print(f"  Quality Score: {quality.get('quality_score', 0):.2f}")
        print(f"  Syntax Valid: {'✓' if quality.get('syntax_valid') else '✗'}")
        print(f"  Security Issues: {len(quality.get('security_issues', []))}")
        print(f"  Overall: {'✓ SUCCESS' if results.get('success') else '✗ FAILED'}")
        
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️  No CPP files found")


In [ ]:
print("\n" + "="*60)
print("TEST SUMMARY")
print("="*60)
print(f"\nC Files Found: {len(c_files)}")
print(f"CPP Files Found: {len(cpp_files)}")
print(f"\nAPI Key Set: {'✓' if os.environ.get('MISTRAL_API_KEY') and os.environ.get('MISTRAL_API_KEY') != 'your-mistral-api-key-here' else '✗'}")
print(f"Tree-sitter Built: {'✓' if os.path.exists('build/my-languages.so') or 'C_LANGUAGE' in globals() else '✗'}")
print(f"Repository: {'✓' if os.path.exists('/kaggle/working/LLMalMorph') else '✗'}")
print("\n" + "="*60)
